In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


 # Demo準備

In [1]:
%%bash
sudo docker rm -f sql19rc1
sudo docker rm -f sql17cu16
sudo docker rm -f registry
sudo docker rmi localhost:5000/mssql/devdb:v1
sudo rm -r ~/SQL_Server_Container/DataVolume/*
sudo rm -r ~/SQL_Server_Container/Dockerfile
echo Cleanup


sql19rc1
registry
Untagged: localhost:5000/mssql/devdb:v1
Untagged: localhost:5000/mssql/devdb@sha256:c022770090779809b26180dc582b8f0012f732ae5c1fbf12ac34ea05ed299a25
Deleted: sha256:89e9180306d51b629179d33d6a0ea744bbb55b2351db313950fbfddf0d3608e0
Deleted: sha256:a07b2e7e8c00f53d30136450bc9665fd847c25618e9089f739274949217d1439
Cleanup
Error: No such container: sql17cu16


 # Demo 1: SQL Server コンテナの実行
 ## 1-1.SQL Server 2017イメージをPull（事前にPull済み）

In [2]:
%%bash
sudo docker pull mcr.microsoft.com/mssql/server:2017-CU16
sudo docker image list | grep 2017-CU16


2017-CU16: Pulling from mssql/server
Digest: sha256:e235aaa983f2ab339de6b7846e93080ad0351cd553cd32d31d5232ccef3a7cbf
Status: Image is up to date for mcr.microsoft.com/mssql/server:2017-CU16
mcr.microsoft.com/mssql/server:2017-CU16
mcr.microsoft.com/mssql/server   2017-CU16           d9b9b96627b7        2 months ago        1.36GB


 ## 1-2.SQL Server 2017コンテナ起動

In [3]:
%%bash
sudo docker run -e ACCEPT_EULA=Y -e MSSQL_SA_PASSWORD=$PASSWORD -p 11433:1433 -v ~/SQL_Server_Container/DataVolume:/var/opt/mssql --name sql17cu16 -d mcr.microsoft.com/mssql/server:2017-CU16
sudo docker ps -a --filter name=sql17cu16
sleep 30


4400de25de4880c3d83bd7d0f362a2d228f55a5883ea9a8410af29f5ab4acf4f
CONTAINER ID        IMAGE                                      COMMAND                  CREATED             STATUS                  PORTS                     NAMES
4400de25de48        mcr.microsoft.com/mssql/server:2017-CU16   "/opt/mssql/bin/sqls…"   2 seconds ago       Up Less than a second   0.0.0.0:11433->1433/tcp   sql17cu16


 ## 1-3.コンテナ内でsqlcmdを実行（コンテナ内は1433ポートでリッスン）

In [4]:
%%bash
sudo docker exec sql17cu16 /opt/mssql-tools/bin/sqlcmd -S localhost,1433 -U sa -P $PASSWORD -Q 'select @@version'



------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2017 (RTM-CU16) (KB4508218) - 14.0.3223.3 (X64) 
	Jul 12 2019 17:43:08 
	Copyright (C) 2017 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS)                                                                                                            

(1 rows affected)


 ## 1-4.コンテナ外でsqlcmd（ホストの11433ポートはコンテナの1433にマッピング）

In [5]:
%%bash
sqlcmd -S localhost,11433 -U sa -P $PASSWORD -Q 'select @@version'



------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2017 (RTM-CU16) (KB4508218) - 14.0.3223.3 (X64) 
	Jul 12 2019 17:43:08 
	Copyright (C) 2017 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS)                                                                                                            

(1 rows affected)


 ## 1-5.DevDB作成

In [6]:
%%bash
sqlcmd -S localhost,11433 -U sa -P $PASSWORD -Q 'create database DevDB'
sqlcmd -S localhost,11433 -U sa -P $PASSWORD -Q 'use DevDB; create table tab01(id int, name varchar(max)); insert into tab01 values (1, "SQL Server runs on Docker!")'
sqlcmd -S localhost,11433 -U sa -P $PASSWORD -Q 'select name from sys.databases; use DevDB; select * from tab01'


Changed database context to 'DevDB'.

(1 rows affected)
name                                                                                                                            
--------------------------------------------------------------------------------------------------------------------------------
master                                                                                                                          
tempdb                                                                                                                          
model                                                                                                                           
msdb                                                                                                                            
DevDB                                                                                                                           

(5 rows affected)
Changed database conte

 ## 1-6.コンテナ内にDBファイルが作成された

In [7]:
%%bash
sudo docker exec sql17cu16 ls -la /var/opt/mssql/data | grep DevDB


-rw-r----- 1 root root  8388608 Oct 26 04:08 DevDB.mdf
-rw-r----- 1 root root  8388608 Oct 26 04:08 DevDB_log.ldf


 ## 1-7.ホストボリュームからもDBファイルが確認できる

In [8]:
%%bash
ls -la ~/SQL_Server_Container/DataVolume/data | grep DevDB


-rw-r----- 1 root root  8388608 Oct 26 04:08 DevDB.mdf
-rw-r----- 1 root root  8388608 Oct 26 04:08 DevDB_log.ldf


 ## 1-8.コンテナ停止

In [9]:
%%bash
sudo docker stop sql17cu16
sudo docker ps -a --filter name=sql17cu16


sql17cu16
CONTAINER ID        IMAGE                                      COMMAND                  CREATED             STATUS                              PORTS               NAMES
4400de25de48        mcr.microsoft.com/mssql/server:2017-CU16   "/opt/mssql/bin/sqls…"   2 minutes ago       Exited (0) Less than a second ago                       sql17cu16


 ## 1-9.コンテナ停止してもホストボリューム上のDBファイルは消えない

In [10]:
%%bash
ls -la ~/SQL_Server_Container/DataVolume/data | grep DevDB


-rw-r----- 1 root root  8388608 Oct 26 04:08 DevDB.mdf
-rw-r----- 1 root root  8388608 Oct 26 04:08 DevDB_log.ldf


 ## 1-10.コンテナ破棄

In [11]:
%%bash
sudo docker rm -f sql17cu16
sudo docker ps -a --filter name=sql17cu16


sql17cu16
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


 ## 1-11.コンテナ破棄してもホストボリューム上のDBファイルは消えない

In [12]:
%%bash
ls -la ~/SQL_Server_Container/DataVolume/data | grep DevDB


-rw-r----- 1 root root  8388608 Oct 26 04:08 DevDB.mdf
-rw-r----- 1 root root  8388608 Oct 26 04:08 DevDB_log.ldf
